In [164]:
from pyqumo.simulations.networks.model import simulate_gg1, simulate_gg1_tandem
from pyqumo.randoms import Poisson, Exponential, Distribution
import pandas as pd
import numpy as np
from dataclasses import dataclass
import random 
from tqdm.notebook import tqdm

In [165]:
results = simulate_gg1_tandem(
    arrivals = Exponential(100),
    services = [Exponential(100)] *  3,
    queue_capacity = 10,
)
print(results.__dict__)
print(results.tabulate())

{'_num_stations': 3, 'real_time': 144.0, 'system_size': [(Countable: p=[0.0858, 0.0861, 0.0858, 0.0853, 0.0849, 0.0834, 0.083, 0.0814, 0.0805, 0.0815, 0.0818, 0.0804]), (Countable: p=[0.122, 0.113, 0.108, 0.101, 0.0961, 0.0899, 0.0812, 0.0729, 0.0642, 0.0551, 0.0492, 0.0471]), (Countable: p=[0.15, 0.134, 0.12, 0.107, 0.0942, 0.0845, 0.073, 0.0643, 0.0554, 0.0465, 0.0386, 0.0317])], 'queue_size': [(Countable: p=[0.172, 0.0858, 0.0853, 0.0849, 0.0834, 0.083, 0.0814, 0.0805, 0.0815, 0.0818, 0.0804]), (Countable: p=[0.235, 0.108, 0.101, 0.0961, 0.0899, 0.0812, 0.0729, 0.0642, 0.0551, 0.0492, 0.0471]), (Countable: p=[0.284, 0.12, 0.107, 0.0942, 0.0845, 0.073, 0.0643, 0.0554, 0.0465, 0.0386, 0.0317])], 'busy': [(Countable: p=[0.0858, 0.914]), (Countable: p=[0.122, 0.878]), (Countable: p=[0.15, 0.85])], 'drop_prob': [0.08129, 0.04534024232263964, 0.030185537854511866], 'delivery_prob': [0.8505675851377706, 1.0, 1.0], 'departures': [Statistics(avg=0.010881884436522481, var=0.000117346504936387

# Cycle for data generation:


In [ ]:
from typing import List
from datetime import datetime
from functools import reduce

@dataclass
class MM1TandemInputData:
    arrivals: List[Distribution]
    services: List[Distribution]
    queue_capacity: int = 10
    max_packets: int = int(1e5)

def extract_experiment_results(input_data, num_stations, results):
    return  {
        "arrivals_exponential_rate":    ";".join([str(arrival.param) for arrival in input_data.arrivals]),

        "services_exponential_rate":    ";".join([str(service.param) for service in input_data.services]),
        
        "queue_capacity":               input_data.queue_capacity,

        "number_of_stations":           num_stations,

        "queue_size_avg":               np.mean([node_queue_size.mean for node_queue_size in results.queue_size]),

        "end_to_end_delay_avg":         sum([node_delivery_delays.avg for node_delivery_delays in results.delivery_delays]),

        "drop_probabilities":           1 - reduce(
                                                lambda x, y: x * y, 
                                                [ (1 - node_drop_prob) for node_drop_prob in results.drop_prob ]
                                            )
    }

def start_experiment(arrival_rate_min, arrival_rate_max, arrival_rate_step,
                     service_rate_min, service_rate_max, service_rate_step,
                     num_stations_min, num_stations_max, num_stations_step,
                     queue_capacity, max_packets, service_dist = None,
                     arrival_dist = None, save_name=""):
    
    experiment_df = pd.DataFrame(columns=["arrivals_exponential_rate", "services_exponential_rate",
                                          "queue_capacity", "number_of_stations", "queue_size_avg", 
                                          "end_to_end_delay_avg", "drop_probabilities"])

    total_num_iterations = ((arrival_rate_max - arrival_rate_min) / arrival_rate_step) * \
                           ((service_rate_max - service_rate_min) / service_rate_step) * \
                           ((num_stations_max - num_stations_min) / num_stations_step)

    with tqdm(total=total_num_iterations) as pbar:
        for num_stations in range(num_stations_min, num_stations_max, num_stations_step):
            for arrival_rate in range(arrival_rate_min, arrival_rate_max, arrival_rate_step):
                for service_rate in range(service_rate_min, service_rate_max, service_rate_step):

                    input_data = MM1TandemInputData(
                        [Exponential(arrival_rate) for _ in range(num_stations)],
                        [Exponential(service_rate) for _ in range(num_stations)],
                        queue_capacity,
                        max_packets
                    )

                    # print(f"Start simulation with input: {input_data.__dict__}")

                    results = simulate_gg1_tandem(**input_data.__dict__)

                    experiment_df = pd.concat([
                            experiment_df, 
                            pd.DataFrame([extract_experiment_results(input_data, num_stations, results)], columns=experiment_df.columns)
                        ], 
                        ignore_index=True
                    )

                    # Raises future warning
                    # experiment_df  = experiment_df.append(
                    #     extract_experiment_results(input_data, num_stations, results), 
                    #     ignore_index=True
                    # )

                    pbar.update(1)
    
    save_name = "experiment-data/" + save_name + datetime.today().strftime('-%Y-%m-%d-%H-%M-%S') + ".csv"
    experiment_df.to_csv(save_name)
    return experiment_df, save_name
        

In [ ]:
df, save_name = start_experiment(100, 1000, 10,  # arrivals
                                 100, 1000, 10,  # services
                                 5,   20,   1,   # num_stations
                                 500, int(1e5),
                                 Exponential,
                                 Exponential,
                                 "experiment-various-num-stations")

# Decision tree

In [ ]:
RANDOM_STATE=42

In [ ]:
def decode_list_from_str(str):
    return list(map(float, str.split(';')))

def get_learning_data_from_dataframe(df):
    X = np.array([[]]).reshape(-1, 3)
    Y = np.array([[]]).reshape(-1, 3)

    for idx, row in df.iterrows():
        x_row = [row["number_of_stations"]] + [decode_list_from_str(row["services_exponential_rate"])[0]] + [decode_list_from_str(row["arrivals_exponential_rate"])[0]]

        X = np.append(X, [x_row], axis=0)
        
        y_row = [row["queue_size_avg"]] + [row["end_to_end_delay_avg"]] + [row["drop_probabilities"]]
        Y = np.append(Y, [y_row], axis=0)

    return X, Y
    

In [ ]:
df = pd.read_csv("experiment-data/experiment-various-num-stations-2024-05-12-03-27-51.csv")

X, Y = get_learning_data_from_dataframe(df)

print(X.shape, Y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=RANDOM_STATE)
print(f"Train size: {X_train.shape[0]}\nTest size: {X_test.shape[0]}")

In [166]:
from sklearn import tree
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

models = [
    tree.DecisionTreeRegressor(random_state=RANDOM_STATE), 
    MultiOutputRegressor(Ridge(random_state=RANDOM_STATE)), 
    RandomForestRegressor(random_state=RANDOM_STATE), 
    MultiOutputRegressor(GradientBoostingRegressor(random_state=RANDOM_STATE)), 
    # MultiOutputRegressor(BaggingRegressor(estimator=SVR(random_state=RANDOM_STATE))),
    # MLPRegressor(random_state=RANDOM_STATE, max_iter=5000),
    
]

for model in tqdm(models):
    model.fit(X_train, Y_train)


  0%|          | 0/4 [00:00<?, ?it/s]

In [186]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

for model in tqdm(models):
    prediction = model.predict(X_test)
    r2_total = r2_score(Y_test, prediction)
    mse_total = mean_squared_error(Y_test, prediction)

    print(f"Model: {model}\nR^2 total\t=\t{r2_total}\nmse_total\t=\t{mse_total}")

    idx_to_label = {
        0: "queue size avg.",
        1: "end to end delay avg",
        2: "drop proba"
    }

    for idx in range(0, 3):
        r2_idx = r2_score(Y_test[idx,:], prediction[idx, :])
        mse_idx = mean_squared_error(Y_test[idx, :], prediction[idx, :])

        print(f"r2_[{idx_to_label[idx]}] = {r2_idx}")
        print(f"mse_[{idx_to_label[idx]}] = {mse_idx}")
    print()
    

  0%|          | 0/4 [00:00<?, ?it/s]

Model: DecisionTreeRegressor(random_state=42)
R^2 total	=	0.9796358303410923
mse_total	=	28.38899393972575
r2_[queue size avg.] = 0.9970093801827589
mse_[queue size avg.] = 124.08277544768244
r2_[end to end delay avg] = 0.9996166577913959
mse_[end to end delay avg] = 18.880573274771248
r2_[drop proba] = 0.9999407245794717
mse_[drop proba] = 2.581426552862249

Model: MultiOutputRegressor(estimator=Ridge(random_state=42))
R^2 total	=	0.4872551765611264
mse_total	=	877.7801766394674
r2_[queue size avg.] = 0.9961597741098922
mse_[queue size avg.] = 159.33348800925185
r2_[end to end delay avg] = 0.996411341177448
mse_[end to end delay avg] = 176.75052299632114
r2_[drop proba] = 0.9819371980847267
mse_[drop proba] = 786.6295349336134

Model: RandomForestRegressor(random_state=42)
R^2 total	=	0.9890150342928951
mse_total	=	14.823511924314362
r2_[queue size avg.] = 0.9981597553577827
mse_[queue size avg.] = 76.35295579620178
r2_[end to end delay avg] = 0.9999177538482681
mse_[end to end delay 

# Prediction

In [ ]:
def predict(model, 
            num_stations, 
            services_exponential_rate, 
            arrivals_exponential_rate):
    
    input_X = np.array([[num_stations, services_exponential_rate, arrivals_exponential_rate]]).reshape(-1, 3)

    predicted_Y = model.predict(input_X)[0]

    predicted_dict = {
        "queue_size_avg":       predicted_Y[0],
        "end_to_end_delay_avg": predicted_Y[1],
        "drop_probability":     predicted_Y[2],
    }

    return predicted_dict


In [ ]:
model = models[2] # RandomForestRegressor

predict(model, 7, 1000, 400)

# Plots


In [ ]:
df = pd.read_csv("experiment-data/experiment-various-num-stations-2024-05-12-03-27-51.csv", index_col=0)

In [ ]:
df

In [181]:
import os

def plot_prediction(df, model, arrivals_rate, services_rate):
    fiexd_rates = df[(df["arrivals_exponential_rate"].map(lambda row: decode_list_from_str(row)[0]) == arrivals_rate) &
                     (df["services_exponential_rate"].map(lambda row: decode_list_from_str(row)[0]) == services_rate)]

    num_stations = fiexd_rates["number_of_stations"].to_numpy()
    
    queue_size_avg = fiexd_rates["queue_size_avg"].to_numpy()

    delays = fiexd_rates["end_to_end_delay_avg"].to_numpy()
    drop_probas = fiexd_rates["drop_probabilities"].to_numpy()

    arrivals_rates = np.array([arrivals_rate] * len(num_stations))
    services_rates = np.array([services_rate] * len(num_stations))
    

    model_input = np.array([num_stations, services_rates, arrivals_rates]).reshape(-1, 3)


    prediction = model.predict(model_input)

    predicted_queue_sizes = prediction[:, 0]
    predicted_delays = prediction[:, 1]
    predicted_drop_probas = prediction[:, 2]

    data_to_plot = [(queue_size_avg, predicted_queue_sizes, "queue size avg."),
                    (delays, predicted_delays, "end to end delay"),
                    (drop_probas, predicted_drop_probas, "drop probability")]

    for real_data, predicted_data, label in data_to_plot:
        fig = plt.figure()
        plt.grid()
        plt.plot(num_stations, real_data, label="real")
        plt.plot(num_stations, predicted_data, label="predicted")
        plt.xlabel("Num stations")
        plt.ylabel(label)
        plt.legend()
        plt.title(f"Arrivals rate = {arrivals_rate}\nServices rate = {services_rate}")
        os.makedirs(f"graphs/{model}/{label}", exist_ok=True)
        plt.savefig(f"graphs/{model}/{label}/arrival={arrivals_rate};service={services_rate}.png")
        plt.close()
    

In [184]:
for model in models:
    for arrival_rate in tqdm(range(100, 1000, 100)):
        for service_rate in range(arrival_rate, 1000, 100):
            plot_prediction(df, model, arrival_rate, service_rate)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]